<a href="https://colab.research.google.com/github/kaitz2981-ux/TTNT-Tuan03/blob/main/Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bài 1